---

# 📡 AI-Based MANET Link Stability Prediction System

---

## 📘 1 — Project Introduction

### 📡 AI-Based MANET Link Stability Prediction System

This project provides:

* 📶 **Mobile Ad-hoc Network (MANET) Link Stability Prediction**
* 🧠 **Deep Learning based Binary Classification**
* ⚡ **Feed-Forward Neural Network using TensorFlow/Keras**
* 📊 **Accuracy, Confusion Matrix & Classification Report**
* 🧪 **Synthetic Dataset Expansion for Robust Training**
* 🌍 **Real-Time Web Deployment using Flask + ngrok**

### ✅ Supported Classes

* **Stable Link (1)**
* **Unstable Link (0)**

### 📌 Input Features

* Distance between nodes (meters)
* Speed of Node 1 (m/s)
* Speed of Node 2 (m/s)
* Link Duration (seconds)
* RSSI (dBm)

---

### 📘 This Notebook Performs

1. Environment & Dependency Setup
2. NS-3 MANET Dataset Loading
3. Large-Scale Dataset Expansion (Synthetic Augmentation)
4. Dynamic Label Recalculation (Link Stability Logic)
5. Train/Test Split for Unseen MANET Scenarios
6. Deep Learning Model Training
7. Model Saving & Evaluation
8. Flask Web App Creation
9. Public Deployment using ngrok

---

## 📘 2 — Imports & Environment Setup

This step:

* Imports all required libraries
* Sets NumPy and TensorFlow random seeds
* Prepares the execution environment

# ===============================

# ✅ CELL 1: Imports & Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import joblib

np.random.seed(42)
tf.random.set_seed(42)

print("Environment ready")


## 📘 3 — Dataset Loading (NS-3 MANET Dataset)

This step:

* Mounts Google Drive
* Loads NS-3 generated MANET dataset
* Displays dataset shape and sample records

### 📌 Dataset Description

The dataset contains simulated MANET link parameters generated using **NS-3**, commonly used for network research.

# ===============================

# ✅ CELL 2: Mount Drive & Load Dataset

# ===============================

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Sasi Projects/ns3_manet_link_dataset.csv")
print("Original dataset shape:", df.shape)
df.head()



## 📘 4 — Dataset Expansion (Critical Step)

This step:

* Artificially expands the dataset (×150)
* Adds realistic Gaussian noise to features
* Improves model generalization
* Prevents overfitting on small NS-3 datasets

### ⚠️ Why This Step Is Important?

NS-3 datasets are often small. Deep learning models **require large-scale data**, so synthetic expansion is essential for stable learning.

# ===============================

# ✅ CELL 3: Dataset Expansion

# ===============================

In [ ]:
expanded_df = pd.concat([df] * 150, ignore_index=True)

expanded_df["distance"] += np.random.normal(0, 6, len(expanded_df))
expanded_df["speed_node1"] += np.random.normal(0, 0.4, len(expanded_df))
expanded_df["speed_node2"] += np.random.normal(0, 0.4, len(expanded_df))
expanded_df["link_duration"] += np.random.normal(0, 3, len(expanded_df))
expanded_df["rssi"] += np.random.normal(0, 1.5, len(expanded_df))

expanded_df["distance"] = expanded_df["distance"].clip(lower=0)
expanded_df["speed_node1"] = expanded_df["speed_node1"].clip(lower=0)
expanded_df["speed_node2"] = expanded_df["speed_node2"].clip(lower=0)
expanded_df["link_duration"] = expanded_df["link_duration"].clip(lower=0)
# ❗ RSSI NOT clipped (keep negative)

print("Expanded dataset shape:", expanded_df.shape)
expanded_df.head()


## 📘 5 — Recompute Link Stability Labels (Most Important Fix)

This step:

* **Recomputes `link_stability` dynamically**
* Ensures labels match physical MANET behavior
* Prevents data leakage and label inconsistency

### 🧠 Stability Logic Used

A link is considered **stable** if:

* Distance < 200 meters
* Node speeds < 8 m/s
* Link duration > 50 seconds
* RSSI > −70 dBm

# ===============================

# ✅ CELL 4: Recompute Labels

# ===============================

In [ ]:
expanded_df["link_stability"] = (
    (expanded_df["distance"] < 200) &
    (expanded_df["speed_node1"] < 8) &
    (expanded_df["speed_node2"] < 8) &
    (expanded_df["link_duration"] > 50) &
    (expanded_df["rssi"] > -70)
).astype(int)

expanded_df["link_stability"].value_counts()


## 📘 6 — Train / Test Split (Unseen MANET Scenarios)

This step:

* Splits data based on **distance**
* Ensures testing on **unseen long-distance links**
* Simulates real-world MANET behavior

### 🔬 Strategy Used

* **Train:** distance ≤ 250 meters
* **Test:** distance > 250 meters

# ===============================

# ✅ CELL 5: Train/Test Split

# ===============================

In [ ]:
# ===============================
# Unseen MANET Scenario Split
# ===============================

train_df = expanded_df[expanded_df["distance"] <= 250]
test_df  = expanded_df[expanded_df["distance"] > 250]

print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

X_train = train_df.drop("link_stability", axis=1)
y_train = train_df["link_stability"]

X_test = test_df.drop("link_stability", axis=1)
y_test = test_df["link_stability"]


## 📘 7 — Feature Scaling

This step:

* Applies **StandardScaler**
* Ensures stable gradient updates
* Prevents dominance of large-valued features

✔ Scaler is saved for real-time inference

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)



## 📘 8 — Deep Learning Model Architecture

This step defines:

* Fully Connected Neural Network
* ReLU activation layers
* Dropout for regularization
* Sigmoid output for binary classification

### 🧠 Architecture Summary

| Layer   | Units       |
| ------- | ----------- |
| Dense   | 16          |
| Dropout | 0.4         |
| Dense   | 8           |
| Output  | 1 (Sigmoid) |

# ===============================

# ✅ CELL 6: Model Definition

# ===============================


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(
        16, activation='relu',
        input_shape=(X_train.shape[1],)
    ),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


## 📘 9 — Model Training

This step:

* Trains the model for 50 epochs
* Uses validation split for monitoring
* Learns nonlinear MANET link behavior

# ===============================

# ✅ CELL 7: Train Model

# ===============================

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


## 📘 10 — Save Trained Model & Scaler

This step:

* Saves trained `.h5` model
* Saves feature scaler using `joblib`
* Enables deployment & reuse

# ===============================

# ✅ CELL 8: Save Model

# ===============================

In [ ]:
model.save("manet_link_stability_model.h5")
joblib.dump(scaler, "scaler.save")

print("✅ Model and scaler saved correctly")

## 📘 11 — Model Evaluation & Metrics

This step evaluates:

* Accuracy
* Precision, Recall, F1-score
* Confusion Matrix

✔ Ensures robust performance on unseen MANET scenarios

# ===============================

# ✅ CELL 9: Evaluation

# ===============================


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))



## 📘 12 — Local Sample Testing

This step:

* Tests model on a handcrafted MANET scenario
* Displays confidence score
* Validates real-world usability

# ===============================

# ✅ CELL 10: Sample Test

# ===============================

In [ ]:
sample = pd.DataFrame(
    [[80, 2, 3, 110, -48]],
    columns=["distance","speed_node1","speed_node2","link_duration","rssi"]
)

sample_scaled = scaler.transform(sample)
prob = model.predict(sample_scaled)[0][0]

print("Prediction:",
      "LINK IS STABLE" if prob > 0.5 else "LINK IS UNSTABLE",
      "Confidence:", round(prob*100,2))

## 📘 13 — Flask Web Application (Real-Time Prediction)

This step:

* Builds a Flask backend
* Accepts MANET parameters from UI
* Returns stability prediction in real-time

### 🌐 Features

* User-friendly input form
* Instant prediction
* Scaler + model integration

# ===============================

# ✅ Flask App: app.py

# ===============================

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

app = Flask(__name__)

# Load model & scaler
model = tf.keras.models.load_model("manet_link_stability_model.h5")
scaler = joblib.load("scaler.save")

@app.route("/", methods=["GET", "POST"])
def home():
    prediction = None

    if request.method == "POST":
        distance = float(request.form["distance"])
        speed1 = float(request.form["speed1"])
        speed2 = float(request.form["speed2"])
        duration = float(request.form["duration"])
        rssi = float(request.form["rssi"])

        data = pd.DataFrame(
            [[distance, speed1, speed2, duration, rssi]],
            columns=["distance", "speed_node1", "speed_node2", "link_duration", "rssi"]
        )

        data_scaled = scaler.transform(data)
        pred = model.predict(data_scaled)[0][0]

        prediction = "LINK IS STABLE" if pred > 0.5 else "LINK IS UNSTABLE"

    return render_template("index.html", prediction=prediction)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


## 📘 14 — Frontend UI (HTML & CSS)

This step:

* Creates clean UI using HTML + CSS
* Dark-themed professional design
* Responsive input form

# ===============================

# ✅ UI Files

# ===============================

* `templates/index.html`
* `static/style.css`


---

In [ ]:
!mkdir -p templates
!mkdir -p static


In [ ]:
!rm -rf templates/index.html

In [ ]:
!ls templates

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>MANET Link Stability Prediction</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>

<div class="container">
    <h1>📡 MANET Link Stability Prediction</h1>
    <p>Enter MANET link parameters to predict stability</p>

    <form method="POST">
        <input type="number" step="any" name="distance" placeholder="Distance (meters)" required>
        <input type="number" step="any" name="speed1" placeholder="Node 1 Speed (m/s)" required>
        <input type="number" step="any" name="speed2" placeholder="Node 2 Speed (m/s)" required>
        <input type="number" step="any" name="duration" placeholder="Link Duration (seconds)" required>
        <input type="number" step="any" name="rssi" placeholder="RSSI (dBm)" required>

        <button type="submit">🔍 Predict Link</button>
    </form>

    {% if prediction %}
    <div class="result">
        <h2>Result: {{ prediction }}</h2>
    </div>
    {% endif %}
</div>

</body>
</html>


In [ ]:
!mkdir -p templates static

In [ ]:
%%writefile static/style.css
body {
    font-family: Arial, sans-serif;
    background: #0f172a;
    color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
}

.container {
    background: #1e293b;
    padding: 30px;
    border-radius: 12px;
    width: 400px;
    text-align: center;
}

input {
    width: 100%;
    padding: 10px;
    margin: 8px 0;
    border-radius: 6px;
    border: none;
}

button {
    width: 100%;
    padding: 12px;
    background: #22c55e;
    color: black;
    font-weight: bold;
    border: none;
    border-radius: 6px;
    cursor: pointer;
}

.result {
    margin-top: 20px;
    background: #16a34a;
    padding: 15px;
    border-radius: 8px;
}


## 📘 15 — Public Deployment using ngrok

This step:

* Launches Flask server
* Creates secure HTTPS public URL
* Enables global access for demo & viva

⚠️ **Ngrok token should not be shared publicly**

In [ ]:
!pkill -f flask || true
!pkill -f ngrok || true

!nohup python app.py > flask.log 2>&1 &

In [ ]:

!pip install pyngrok

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "30okk9QVCXc4jaeD6xQw8iqcsKd_inZ4Fa2tjFE1dhLvi9zm"
public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

## 📘 16 — Final Project Summary

✔ NS-3 MANET Dataset

✔ Deep Learning Classification

✔ Synthetic Data Expansion

✔ Real-Time Prediction

✔ Flask + ngrok Deployment